In [2]:
import os
import csv
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle
from random import randint


samples = []
with open('recovery 2/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

print("samples size:",len(samples))
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)


def generator(samples, batch_size=512):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_name = 'recovery 2/IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(center_name)
                assert center_image is not None
                center_angle = float(batch_sample[3])
                selection = randint(0,5) # can further expend left-flip and right-flip images
                correction = 0.35
                if(selection == 3):#using flip image as argument image
                    images.append(center_image)
                    angles.append(center_angle)
                    images.append(cv2.flip(center_image,1))
                    angles.append(center_angle*-1.0)
                elif(selection == 1):#adding left camera image as argument image
                    images.append(center_image)
                    angles.append(center_angle)
                    left_name = 'recovery 2/IMG/'+batch_sample[1].split('/')[-1]
                    left_image = cv2.imread(left_name)
                    assert left_image is not None
                    left_angle = center_angle + correction
                    images.append(left_image)
                    angles.append(left_angle)
                elif(selection == 2): #adding right camera image as argument image
                    images.append(center_image)
                    angles.append(center_angle)
                    right_name = 'recovery 2/IMG/'+batch_sample[2].split('/')[-1]
                    right_image = cv2.imread(right_name)
                    assert right_image is not None
                    right_angle = center_angle - correction
                    images.append(right_image)
                    angles.append(right_angle)
                elif(selection == 4):# left flip 
                    images.append(center_image)
                    angles.append(center_angle)
                    left_name = 'recovery 2/IMG/'+batch_sample[1].split('/')[-1]
                    left_image = cv2.imread(left_name)
                    assert left_image is not None
                    left_angle = center_angle + correction
                    images.append(cv2.flip(left_image,1))
                    angles.append(left_angle*-1.0)
                elif(selection ==5):#right flip
                    images.append(center_image)
                    angles.append(center_angle)
                    right_name = 'recovery 2/IMG/'+batch_sample[2].split('/')[-1]
                    right_image = cv2.imread(right_name)
                    assert right_image is not None
                    right_angle = center_angle - correction
                    images.append(cv2.flip(right_image,1))
                    angles.append(right_angle*-1.0) 
                
                 
                

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            #print("size of x train:",len(X_train))
            yield shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=512)
validation_generator = generator(validation_samples, batch_size=512)

samples size: 11612


In [3]:
#X_train = X_train.append(recovery_2_images)
#y_train = y_train.append(recovery_2_measurements)

#define network ： Nvidia network

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Convolution2D,MaxPooling2D,Dropout,Cropping2D

ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
model.add(Cropping2D(cropping=((60,20),(0,0)),input_shape=(160,320,3)))
model.add(Lambda(lambda x:x / 255.0 - 0.5,input_shape=(80,320,3),output_shape=(80,320,3)))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))
model.add(Convolution2D(64,3,3,activation="relu"))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
#model.fit(X_train,y_train,validation_split=0.2,shuffle=True,nb_epoch=5)
print ("training sample size:", len(train_samples))
train_lenth = len(train_samples)
print("valid sample size:",len(validation_samples))
model.fit_generator(train_generator, samples_per_epoch = train_lenth*7 , validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=5)
model.save('model_recovery_normal_nvidia.h5')




Using TensorFlow backend.


training sample size: 9289
valid sample size: 2323
Epoch 1/5
64504/65023 [============================>.] - ETA: 6s - loss: 0.1034   

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


65340/65023 [==============================] - 890s - loss: 0.1033 - val_loss: 0.0897
Epoch 2/5
65210/65023 [==============================] - 881s - loss: 0.0850 - val_loss: 0.0834